<a href="https://colab.research.google.com/github/Blind5518/pyspark_films_actors/blob/main/pyspark_films_actors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Загрузка библиотек,  импорт и чтение данных**

In [6]:
# Инициализация Spark
!pip install pyspark
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import DoubleType

spark = SparkSession.builder.appName("MoviesActorsAnalysis").getOrCreate()

In [7]:
# Чтение CSV
movies_df = spark.read.csv("/movies.csv", header=True, inferSchema=False)
actors_df = spark.read.csv("/actors.csv", header=True, inferSchema=False)
movie_actors_df = spark.read.csv("/movie_actors.csv", header=True, inferSchema=False)

**2. Преобразование и создание временных таблиц**

In [8]:
# Преобразования (столбцы дат и числовые)
movies_df = movies_df.withColumn("release_date", F.to_date("release_date", "yyyy-MM-dd"))
actors_df = actors_df.withColumn("birth_date", F.to_date("birth_date", "yyyy-MM-dd"))
movies_df = movies_df.withColumn("budget", F.col("budget").cast(DoubleType()))

# Регистрация временных таблиц
movies_df.createOrReplaceTempView("movies")
actors_df.createOrReplaceTempView("actors")
movie_actors_df.createOrReplaceTempView("movie_actors")

**3. SQL-запросы**

Выполняется через spark.sql("...")

In [9]:
# Топ-5 жанров по количеству фильмов
spark.sql("""select genre,
                    count(movie_id) as num_movies
             from movies
             group by genre
             order by num_movies desc
             limit 5;""").show()

+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+



In [10]:
# Актер с наибольшим количеством фильмов
spark.sql("""select a.name,
                    count(distinct ma.movie_id) as num_movies
             from actors as a join movie_actors as ma using(actor_id)
             group by a.name
             order by num_movies desc
             limit 1;""").show()

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_24|         5|
+--------+----------+



In [11]:
# Cредний бюджет фильмов по жанрам
spark.sql("""select genre,
                    avg(budget) as avg_budget
             from movies
             group by genre
             order by avg_budget desc;""").show()

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
|Horror|      8.7281876775E7|
|Sci-Fi|       7.809715175E7|
| Drama| 6.076021856166667E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
+------+--------------------+



In [13]:
# Фильмы, в которых снялось более одного актера из одной страны
spark.sql("""select m.title,
                    a.country,
                    count(distinct a.actor_id) as num_actors
             from movie_actors ma
             join actors a on ma.actor_id = a.actor_id
             join movies m on ma.movie_id = m.movie_id
             group by m.title, a.country
             having count(distinct a.actor_id) > 1;""").show()

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
|Movie_10|       UK|         2|
|Movie_15|    India|         2|
|Movie_18|Australia|         2|
| Movie_1|    India|         2|
| Movie_7|      USA|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+

